In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.linalg import svd
from functions import *
from get_estimated_covariance import *
import scipy.stats
from scipy.interpolate import UnivariateSpline
import seaborn as sns
import pandas as pd
import tensorflow as tf
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [7]:
sns.set()
sns.set_context('paper')

In [13]:
def plot_data(data):
    for i in range(len(data)):
        plt.plot(data[i])

#  Generating fragments

In [144]:
def generate_fragments(n, K, delta, cov, mu):
    data = np.random.multivariate_normal(mu,cov,n)
    start = np.random.randint(0, int(K*(1-delta)),n)
    end = start + int(delta*K)
    bounds = np.c_[start,end]
    frags = np.empty((n,K))
    frags[:] = np.nan
    for i in range(n):
        s,t = bounds[i]
        frags[i,s:t]=data[i,s:t]
    safe = int(0.05*K*delta)
    safebounds = np.c_[start+safe,end-safe]
    return data,frags,bounds,safebounds

In [145]:
def get_mean(frags):
    return np.nanmean(frags,0)

# Covariance estimation 

In [183]:
res = []

In [184]:
for ite in range(20):
    print(ite)
    n = 500
    K = 50
    delta = 0.5
    t = np.linspace(0,1,K)
    ma = np.meshgrid(t,t)
    cov = np.minimum(ma[0],ma[1])
    eigv, eigf = np.linalg.eigh(cov)
    r = 10
    rA = eigf[:,-r:] @ np.diag(eigv[-r:]) @ eigf[:,-r:].T
    full, frags, bounds, safebounds = generate_fragments(n,K,delta,rA,np.zeros(K))

    estcov_full = np.cov(full, rowvar=False)
    estcov_frags = get_trunc_cov(full,safebounds)

    svd = np.linalg.svd(estcov_frags)
    P = get_P(K,0.9*delta)
    res = []
    def get_best_param(i):
        init_gamma = svd[0][:,:i] @ np.diag(svd[1][:i])

        tf.reset_default_graph()
        tf.set_random_seed(42)
        Rr = tf.constant(estcov_frags, dtype=tf.float32, name = "R")
        Pp = tf.constant(P, dtype = tf.float32, name = "P")
        in_gamma = tf.placeholder(dtype=tf.float32,shape = (K,i), name='ini')
        gamma = tf.Variable(in_gamma + tf.random_normal([K,i],0,1),dtype=tf.float32, name = "gamma")
        pred = tf.multiply(Pp,tf.matmul(gamma,gamma,transpose_b=True))
        error = Rr - pred
        mse = tf.reduce_mean(tf.square(error), name="mse")
        optimizer = tf.contrib.opt.ScipyOptimizerInterface(mse,options={'maxiter': 1000000, 'gtol':1e-50,'ftol':1e-50})
        init = tf.global_variables_initializer()
        with tf.Session() as sess:
            sess.run(init,feed_dict={in_gamma:init_gamma.astype(np.float32)})
            optimizer.minimize(sess)
            best_gamma = gamma.eval()
        return best_gamma @ best_gamma.T

    estcov_recov = get_best_param(r)

    bds = np.copy(bounds)
    bds[np.argmin(bds,0)[0],0]=0;bds[np.argmax(bds,0)[0],1]=K-1
    t = np.linspace(0,1,K)
    data = np.copy(full)

    def extend_to_the_right(data,aux):
        newdata = []
        for i in range(n):
            s,st = bds[i,0],bds[i,1]
            cap = True
            while cap:
                j = np.random.randint(0,n)
                if bds[j,0]<st and bds[j,0]>=s:
                    cap = False
            s1,st1 = bds[j,0],bds[j,1]
            bds[i,1] = st1
            newdata.append(np.append(aux[i],data[j][st:st1] + (aux[i][-1]-data[j][st])))
        return newdata

    def extend_to_the_left(data,aux):
        newdata = []
        for i in range(n):
            s,st = bds[i,0],bds[i,1]
            cap = True
            while cap:
                j = np.random.randint(0,n)
                if bds[j,0]<=s and bds[j,1]>=s:
                    cap = False
            s1,st1 = bds[j,0],bds[j,1]
            bds[i,0] = s1
            newdata.append(np.append(data[j][s1:s] + (aux[i][0]-data[j][s]), aux[i]))
        return newdata

    aux = [data[i,bds[i,0]:bds[i,1]] for i in range(n)]

    for it in range(200):
        aux = extend_to_the_right(data, aux)

    for it in range(200):
        aux = extend_to_the_left(data, aux)


    final = np.zeros((n,min([len(a) for a in aux])))
    for i in range(n):
        s,st = bds[i,0],bds[i,1]
        final[i] = aux[i][:st-s]
    estcov_ext = np.cov(final,rowvar=False)

    err = []
    tres = min(len(estcov_full),len(estcov_ext),len(estcov_recov),len(estcov_frags))
    true_norm = np.linalg.norm(cov[:tres,:tres])
    err.append(np.linalg.norm(cov[:tres,:tres] - estcov_full[:tres,:tres])/true_norm)
    err.append(np.linalg.norm(cov[:tres,:tres] - estcov_frags[:tres,:tres])/true_norm)
    err.append(np.linalg.norm(cov[:tres,:tres] - estcov_recov[:tres,:tres])/true_norm)
    err.append(np.linalg.norm(cov[:tres,:tres] - estcov_ext[:tres,:tres])/true_norm)
    res.append(err)

0
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.004258
  Number of iterations: 184
  Number of functions evaluations: 213
1
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.004859
  Number of iterations: 193
  Number of functions evaluations: 217
2
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.003469
  Number of iterations: 184
  Number of functions evaluations: 206
3
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.004041
  Number of iterations: 201
  Number of functions evaluations: 236
4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.0050